In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import torch
from transformers import AutoTokenizer, AutoModel
from datasets import load_dataset
from tqdm import tqdm
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [4]:
# Загрузка датасета
ds = load_dataset("aglazkova/keyphrase_extraction_russian")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

hist_train.csv:   0%|          | 0.00/4.89M [00:00<?, ?B/s]

ling_train.csv:   0%|          | 0.00/7.97M [00:00<?, ?B/s]

math_train.csv:   0%|          | 0.00/8.18M [00:00<?, ?B/s]

...

ling_test.csv:   0%|          | 0.00/3.72M [00:00<?, ?B/s]

math_test.csv:   0%|          | 0.00/2.44M [00:00<?, ?B/s]

med_test.csv:   0%|          | 0.00/4.15M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20844 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10004 [00:00<?, ? examples/s]

In [5]:
ds

DatasetDict({
    train: Dataset({
        features: ['abstract', 'keywords'],
        num_rows: 20844
    })
    test: Dataset({
        features: ['abstract', 'keywords'],
        num_rows: 10004
    })
})

In [6]:
# Проверка наличия GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Загрузка модели и токенайзера
tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_mt_nlu_ru")
model = AutoModel.from_pretrained("ai-forever/sbert_large_mt_nlu_ru").to(device)

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/866 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

In [7]:
# Функция для получения эмбеддингов с mean pooling
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

In [8]:
ds['test'][0]

{'abstract': 'Статья посвящена научной и общественной деятельности известного российского ученого историка М. И. Капустина. Описан его жизненный и творческий путь. Показана его научная и педагогическая деятельность',
 'keywords': 'иркутский госуниверситет, история гражданской войны в россии, вузовский педагог, участник великой отечественной войны'}

In [9]:
# Предсказание и оценка
abstract_embeddings = []
keywords_embeddings = []

for example in tqdm(ds["test"]):
    # Токенизация и получение эмбеддингов для abstract
    encoded_input = tokenizer(example["abstract"], padding=True, truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
        model_output = model(**encoded_input)
    abstract_embedding = mean_pooling(model_output, encoded_input['attention_mask']).cpu().numpy()
    abstract_embeddings.append(abstract_embedding)

    # Токенизация и получение эмбеддингов для keywords
    encoded_input = tokenizer(example["keywords"], padding=True, truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
        model_output = model(**encoded_input)
    keywords_embedding = mean_pooling(model_output, encoded_input['attention_mask']).cpu().numpy()
    keywords_embeddings.append(keywords_embedding)

100%|██████████| 10004/10004 [08:00<00:00, 20.82it/s]


In [10]:
abstract_embeddings

[array([[ 0.73135835, -0.32384244, -1.1033969 , ...,  0.07431193,
          0.5847879 ,  0.65462357]], dtype=float32),
 array([[ 0.38497862,  0.19168214, -2.5460474 , ..., -0.7074681 ,
          0.5168393 ,  0.49563164]], dtype=float32),
 array([[ 0.19972831,  0.6023598 , -1.7721183 , ..., -1.2448325 ,
          1.005413  ,  1.0584124 ]], dtype=float32),
 array([[ 0.39470145,  0.20707086, -0.90764046, ..., -1.2350227 ,
          1.4553277 ,  0.9023027 ]], dtype=float32),
 array([[ 0.82328135, -0.20331465, -0.13263205, ..., -0.19911571,
          1.0636703 ,  0.75734675]], dtype=float32),
 array([[ 0.8602854, -0.3396701, -1.6155108, ..., -0.1373476,  1.4116627,
          1.0554216]], dtype=float32),
 array([[ 9.5089662e-01, -9.7827043e-04, -1.0795548e+00, ...,
         -5.8573925e-01,  1.1016743e+00,  1.1063491e+00]], dtype=float32),
 array([[ 1.1559722 ,  0.08308743, -0.6349494 , ..., -0.3526043 ,
          1.1594747 ,  0.7514879 ]], dtype=float32),
 array([[ 0.6873154 , -0.01049498, -

In [11]:
# Конвертация списков эмбеддингов в тензоры
abstract_embeddings = torch.tensor(abstract_embeddings).squeeze()
keywords_embeddings = torch.tensor(keywords_embeddings).squeeze()

<ipython-input-11-f5f166a35f27>:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  abstract_embeddings = torch.tensor(abstract_embeddings).squeeze()


In [14]:
abstract_embeddings.shape

torch.Size([10004, 1024])

In [13]:
# Метрики: MSE, MAE и R2
mse = mean_squared_error(abstract_embeddings, keywords_embeddings)
mae = mean_absolute_error(abstract_embeddings, keywords_embeddings)
r2 = r2_score(abstract_embeddings, keywords_embeddings)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R2):", r2)

Mean Squared Error (MSE): 0.24541609
Mean Absolute Error (MAE): 0.3730642
R-squared (R2): -0.4250143475426355
